In [1]:
import json
import pandas as pd
import codecs
from pandas.io.json import json_normalize as jsy
#!pip install gensim
from gensim.models import Word2Vec
from nltk.corpus import brown # , #movie_review
import numpy as np
from textblob import TextBlob as tb

C:\Users\gokul\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dev_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-dev.json', lines=True)
train_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-train.json', lines = True)
test_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-test.json', lines = True)

In [107]:
train_set.iloc[999]


article                                                    Physics
candidates                                                     [3]
is_paraphrase                                                 True
question         What's an example of something that was a prob...
section                                             Modern physics
sentences        [Modern physics began in the early 20th centur...
type                                                       SCIENCE
Name: 999, dtype: object

### following is helper cell no need to run for exposure purpose

In [ ]:
test_set[test_set['article']=='Table tennis']
x = test_set.loc[1422]['sentences']
train_set[(train_set['article']=='Table tennis')&(train_set['section']=='Ball')]

temp = (train_set.loc[0]['sentences'])
j=0
for i in range(len((train_set.loc[0]['sentences']))):
    print(j)
    print(temp[i])
    j+=1

### Create new dataframe from origine - mapping question to answer yet to complete for more than 1 candidate keys

In [4]:
train = pd.DataFrame(train_set['question'])
train['answers'] = "NA"
train.shape

temp1 = len(train_set)
for i in range(temp1):
    #print(type(train_set.loc[i]['question']))
    temp = len(train_set.loc[i]['candidates'])
    if temp==1:
        anskey = train_set.loc[i]['candidates'][0]
        #print(type(train_set.loc[i]['sentences'][anskey]))
        train.loc[i]['answers'] = train_set.loc[i]['sentences'][anskey]
    else:
        pass

In [139]:
writetrain = train[train['answers']!="NA"]
X_train = np.array(writetrain['question'])
Y_train = np.array(writetrain['answers'])

maxLenxt = len(max(X_train, key=len).split())
maxLenyt = len(max(Y_train, key=len).split())
minLenxt = len(min(X_train, key=len).split())
minLenyt = len(min(Y_train, key=len).split())
print("Max lenght of X training:", maxLenxt)
print("Max lenght of Y training:", maxLenyt)
print("Min lenght of X training:", minLenxt)
print("Min lenght of Y training:", minLenyt)

Max lenght of X training: 31
Max lenght of Y training: 83
Min lenght of X training: 3
Min lenght of Y training: 1


In [6]:
def read_glove_vec(glove_file):
    with open(glove_file,encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vec('C:/Users/gokul/Documents/Deep_learning/New folder/C5W2A2 - Emojify - v2/data/glove.6B.50d.txt')

word = "expired"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of expired in the vocabulary is 142847
the 289846th word in the vocabulary is potatos


(5079,)

### Removing Question mark in all sentences

In [39]:
count = 0
countn = 0
for i in range(len(X_train)):
    temp = X_train[i][-1]
    if temp != "?":
        pass
    else:
        X_train[i] = X_train[i][:-1]
        temp = X_train[i][-1]
        if temp == "?":
            X_train[i] = X_train[i][:-1]
     
### validating if all "?" are removed
count = 0
countn = 0
for i in range(len(X_train)):
    temp = X_train[i][-1]
    if temp != "?":
        #print(X_train[i])
        count+=1
    else:
        countn+=1
        print(i)
        print(X_train[i])
        #X_train[i] = X_train[i][:-1]
print(count)
print(countn)

5079
0


In [190]:
def sentences_to_indices(X, word_to_vec_map, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len,50))
    te = list(word_to_vec_map.keys())
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        temp1 = tb(X[i])
        temp2 = tb(str(temp1.lower()))
        sentence_words =temp2.words
#         print(sentence_words)
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
#         print(i,j)
#         print(word_to_vec_map['during'])
        for w in sentence_words:
#             print(w)
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            if str(w) in te:
                X_indices[i, j] = word_to_vec_map[str(w)]
            j = j + 1
        
    return X_indices

In [ ]:
#X = sentences_to_indices(X_train, word_to_vec_map, 35)
Y = sentences_to_indices(Y_train, word_to_vec_map, 100)

In [195]:
X.shape

(5079, 35, 50)

In [172]:
X[100,6]

array([ 0.53248  ,  0.030684 , -0.12955  , -0.15673  ,  0.25168  ,
        0.20269  , -0.71869  , -0.27819  , -0.47384  ,  0.49715  ,
       -0.12525  , -0.249    ,  0.23788  ,  0.11087  ,  0.44788  ,
       -0.10767  ,  0.44033  ,  0.16702  , -0.34068  , -0.5413   ,
       -0.56092  , -0.12457  ,  0.23586  ,  0.39872  ,  0.13578  ,
       -1.4765   , -1.1334   , -0.23475  ,  0.17915  ,  0.20182  ,
        3.9566   ,  0.0092012,  0.61391  , -0.75382  ,  0.42119  ,
        0.092947 , -0.2623   ,  0.48914  , -0.78757  , -0.10654  ,
       -0.68392  ,  0.34472  ,  0.4279   ,  0.71161  ,  0.051375 ,
        0.33759  , -0.72084  ,  0.069335 , -0.34333  , -0.067937 ])

In [180]:
word_to_vec_map['diffferent']

KeyError: 'diffferent'

In [184]:

type(te)

list

In [186]:
te[1000]

'themselves'

In [189]:
str('diffferent') in te

False

In [3]:

b = Word2Vec(brown.sents())
b.most_similar('money', topn=5)

C:\Users\gokul\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[('care', 0.9206393361091614),
 ('job', 0.9165905714035034),
 ('trouble', 0.9065046310424805),
 ('work', 0.8878711462020874),
 ('getting', 0.8855317234992981)]

In [4]:
type(brown.sents())

nltk.corpus.reader.util.ConcatenatedCorpusView

In [7]:
x = list(brown.sents())

In [8]:
len(x)

57340

In [9]:
len(x[10])

14

In [10]:
x[10]

['It',
 'urged',
 'that',
 'the',
 'city',
 '``',
 'take',
 'steps',
 'to',
 'remedy',
 "''",
 'this',
 'problem',
 '.']

In [14]:
b.build_vocab_from_freq(word_freq=3)

TypeError: object of type 'int' has no len()